<a href="https://colab.research.google.com/github/Pynthamil/The-Hunger-Games-The-Google-Edition/blob/main/Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install joypy

In [3]:
!pip install pytrends

In [4]:
#import necessary libraries
import pandas as pd
import numpy as np
import joypy
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns

In [5]:
sns.set_theme(style="whitegrid", palette="Spectral")

In [6]:
from pytrends.request import TrendReq

pytrends = TrendReq()
pytrends.build_payload(["social media", "smartphone", "cloud computing", "streaming", "AI"],
                       timeframe="2004-01-01 2023-12-31"
                       )

trends = pytrends.interest_over_time()
trends = trends.drop(columns=["isPartial"])

In [7]:
trends.head()

,social media,smartphone,cloud computing,streaming,AI
date,,,,,
2004-01-01,0,2,0,8,7
2004-02-01,0,2,0,8,7
2004-03-01,0,2,0,8,7
2004-04-01,0,2,0,8,7
2004-05-01,0,2,0,7,7


In [8]:
trends.tail()

,social media,smartphone,cloud computing,streaming,AI
date,,,,,
2023-08-01,6,4,1,30,76
2023-09-01,7,4,1,32,77
2023-10-01,7,4,1,31,88
2023-11-01,7,4,1,31,91
2023-12-01,6,4,1,32,90


In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nharshavardhana/killed-by-google-google-graveyard")

print("Path to dataset files:", path)

100%|██████████| 17.5k/17.5k [00:00<00:00, 9.86MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/nharshavardhana/killed-by-google-google-graveyard/versions/1


In [10]:
#data preparation
#1. Loading dataset
df = pd.read_csv("/kaggle/input/killed-by-google-google-graveyard/Google_graveyard.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/killed-by-google-google-graveyard/Google_graveyard.csv'

In [ ]:
#take a look at the data
df.head()

In [ ]:
df.tail()

In [ ]:
#list the number of rows and columns
rows, columns = df.shape

print("The rows are: ", rows)
print("The columns are: ", columns)

In [ ]:
#checking the launch year vs lifespan
plt.figure(figsize=(10,6))

sns.regplot(
    data=df,
    x="Start",
    y="Total years",
)

plt.title("Has Google’s tolerance for product failure increased over time?")
plt.xlabel("Launch Year")
plt.ylabel("Product Lifespan (years)")

plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))

plt.hexbin(
    df["Start"],
    df["Total years"],
    cmap="summer",
    gridsize=25,
)

plt.colorbar(label="Number of Products")

plt.xlabel("Launch Year")
plt.ylabel("Lifespan (Years)")
plt.title("Did the era a product was born in influence how long it lived?")
plt.show()


In [ ]:
# Distribution of Total Years (Lifespans) by Category
plt.figure(figsize=(10, 6))

joypy.joyplot(
    data=df,
    column='Total years',
    by='Category',
    colormap=plt.cm.summer_r,
    alpha=0.6
)

plt.title('Which categories of Google products are most likely to be discontinued?')
plt.xlabel('Lifespan')
plt.ylabel('Density')
plt.grid(True)
plt.show()

In [ ]:
# Average Google Product Lifespan by Launch Year

avg_lifespan = df.groupby("Start")["Total years"].mean().reset_index()


plt.figure(figsize=(10, 6))
plt.plot(
    avg_lifespan["Start"],
    avg_lifespan["Total years"],
    marker="o"
)
plt.title("Has the average lifespan of Google products declined over time?")
plt.xlabel("Launch Year")
plt.ylabel("Average Lifespan (Years)")
plt.grid(True)

plt.show()

In [ ]:
category_counts = df['Category'].value_counts()
category_counts

In [ ]:
shutdowns = df["End"].value_counts().sort_index()

plt.figure(figsize=(10, 6))

plt.bar(shutdowns.index, shutdowns.values)

plt.title("When Did Google Shut Down the Most Products?")
plt.xlabel("Shutdown Year")
plt.ylabel("Number of Products")

plt.show()

In [ ]:
years = range(df["Start"].min(), df["End"].max() + 1)
active_products_per_year = []

for year in years:
    active_products = df[(df["Start"] <= year) & (df["End"] >= year)]
    active_products_per_year.append(len(active_products))

plt.figure(figsize=(10, 6))
plt.plot(years, active_products_per_year, marker='o')
plt.title("Is Google’s ecosystem becoming more crowded or shrinking? ")
plt.xlabel("Year")
plt.ylabel("Number of Active Products")
plt.grid(True)
plt.show()

In [ ]:
# Google Product Launches Over Time

launch_counts = df["Start"].value_counts().sort_index()

plt.figure(figsize=(10,6))

plt.plot(
    launch_counts.index,
    launch_counts.values,
    marker="o"
)

plt.title("Has Google been experimenting more aggressively over time?")
plt.xlabel("Year")
plt.ylabel("Number of New Products")
plt.grid(True)

plt.show()

In [ ]:
# Count Trends per Year
trends["year"] = trends.index.year

trends_yearly = (
    trends.groupby("year")
    .mean()
    .reset_index()
)

trends_yearly.head()

In [ ]:
# Count shutdowns per Year

shutdowns = df["End"].value_counts().sort_index()

shutdowns = shutdowns.reset_index()
shutdowns.columns = ["year", "shutdown_count"]

shutdowns.head()

In [ ]:
merged = trends_yearly.merge(
    shutdowns,
    on="year",
    how="left"
)

merged["shutdown_count"] = merged["shutdown_count"].fillna(0)

merged.head()

In [ ]:
# Internet Culture vs Google Shutdowns
culture_cols = ["social media", "smartphone", "cloud computing", "streaming", "AI"]

fig, axes = plt.subplots(
    2, 3,
    figsize=(18,10),
    sharex=True,
    sharey=True
)

axes = axes.flatten()

for ax, col in zip(axes, culture_cols):

    ax.bar(
        merged["year"],
        merged["shutdown_count"],
        alpha=0.3,
    )

    ax.set_title(col)

    ax2 = ax.twinx()
    ax2.plot(
        merged["year"],
        merged[col],
        linewidth=2
    )

# remove unused subplot (6th box)
fig.delaxes(axes[-1])

axes[0].set_ylabel("Shutdowns")

plt.suptitle("How have shifts in internet culture influenced the shutdown of Google's products?", fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
corr_matrix = merged.corr(numeric_only=True)

plt.figure(figsize=(8,6))

sns.heatmap(
    corr_matrix,
    annot=True,
    center=0,
    cmap="summer",
)

plt.title("Correlation: Internet Culture vs Google Shutdowns")
plt.show()
